In [240]:
library(Seurat)
library(dplyr)

In [241]:
# Seurat object
mydir = "/share/ScratchGeneral/anncuo/OneK1K/"
sce_filename = paste0(mydir,"cell_type.RDS")

In [242]:
# 57,641 features 
# across 1272489 samples (cells)
# with two assays: SCT and RNA
# dimensionality reductions: PCA, UMAP
sce = readRDS(sce_filename)
sce

An object of class Seurat 
57641 features across 1272489 samples within 2 assays 
Active assay: SCT (24903 features, 3000 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

In [243]:
# Seurat object
mydir = "/share/ScratchGeneral/anncuo/sc-integration-workshop/"
all_meta_filename = paste0(mydir,"all_meta.rds")
all_meta = readRDS(all_meta_filename)
head(all_meta)

,batch_id,celltype.mDC.l1,celltype.mDC.l2,celltype.mDC.l3
,<chr>,<chr>,<chr>,<chr>
10_1016_j_cell_2020_08_001_Schulte-Schrepping-0,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono
10_1016_j_cell_2020_08_001_Schulte-Schrepping-1,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono
10_1016_j_cell_2020_08_001_Schulte-Schrepping-10,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono
10_1016_j_cell_2020_08_001_Schulte-Schrepping-100,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono
10_1016_j_cell_2020_08_001_Schulte-Schrepping-1000,10_1016_j_cell_2020_08_001,CD8 T,CD8 TEM,CD8 TEM_5
10_1016_j_cell_2020_08_001_Schulte-Schrepping-10000,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono


In [244]:
subset_all_meta = all_meta[all_meta$batch_id == "powell_eqtl",]
nrow(subset_all_meta)
head(subset_all_meta,2)

[1] 1247768

,batch_id,celltype.mDC.l1,celltype.mDC.l2,celltype.mDC.l3
,<chr>,<chr>,<chr>,<chr>
powell_eqtl_AAACCTGAGAATGTTG-1,powell_eqtl,CD4 T,CD4 TCM,CD4 TCM_1
powell_eqtl_AAACCTGAGAGAACAG-1,powell_eqtl,NK,NK,NK_1


In [245]:
rownames(subset_all_meta) = gsub("powell_eqtl_","",rownames(subset_all_meta))
head(subset_all_meta,2)

,batch_id,celltype.mDC.l1,celltype.mDC.l2,celltype.mDC.l3
,<chr>,<chr>,<chr>,<chr>
AAACCTGAGAATGTTG-1,powell_eqtl,CD4 T,CD4 TCM,CD4 TCM_1
AAACCTGAGAGAACAG-1,powell_eqtl,NK,NK,NK_1


In [247]:
sce = sce[,colnames(sce) %in% rownames(subset_all_meta)]

In [250]:
dim(sce)

[1]   24903 1244678

In [251]:
subset_all_meta = subset_all_meta[rownames(subset_all_meta) %in% colnames(sce),]

In [252]:
sce@meta.data$celltype.mDC.l2 = subset_all_meta$celltype.mDC.l2

In [253]:
pseudobulk = AggregateExpression(sce, group.by=c("individual","celltype.mDC.l2"))

Warning message in PseudobulkExpression(object = object, pb.method = "aggregate", :
“Exponentiation yielded infinite values. `data` may not be log-normed.”


In [254]:
mat = pseudobulk$RNA
write.csv(mat,"/share/ScratchGeneral/anncuo/sc-integration-workshop/new_celltypes_donor_celltype_summed.csv")

In [26]:
## Covariates
mydir = "/share/ScratchGeneral/anncuo/OneK1K/"
covs_filename =  paste0(mydir,"/covariates.tsv")
df_covs = read.csv(covs_filename, sep="\t")
head(df_covs)

,sampleid,SEX,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1_1,2,0.02775250,-0.000592955,0.00757785,-0.01409290,0.009323330,0.031375400
2,10_10,2,0.02052560,0.040040200,-0.01847820,0.03658380,-0.014617600,0.013474100
3,1000_1001,2,0.01081720,-0.004233600,0.00537891,-0.02058190,-0.027539500,-0.039861100
4,1001_1002,2,0.01211940,0.000788658,-0.00671328,-0.01969960,-0.000180694,0.037322600
5,1002_1003,1,-0.05739470,-0.019243700,0.00442424,0.00619121,-0.010877400,-0.013743000
6,1003_1004,2,0.00532722,0.008837440,-0.01299120,0.00997544,-0.024854800,0.000188687


In [97]:
## Age covariates
age_filename =  paste0(mydir,"/age_covariate.tsv")
df_age = read.csv(age_filename, sep="\t")
colnames(df_age)[1] = "sampleid"
head(df_age)

,sampleid,age
,<chr>,<int>
1,1_1,65
2,2_2,48
3,3_3,53
4,4_4,48
5,5_5,41
6,6_6,79


In [98]:
age_sex = inner_join(df_age, df_covs)
head(age_sex)

Joining, by = "sampleid"


,sampleid,age,SEX,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6
,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1_1,65,2,0.02775250,-0.000592955,0.00757785,-0.014092900,0.00932333,0.03137540
2,2_2,48,2,-0.04813530,-0.056674100,-0.01547420,-0.000729391,0.00027473,0.01116000
3,3_3,53,2,0.01369180,-0.046308100,-0.00584410,0.026565000,-0.00435940,0.03093290
4,4_4,48,2,0.01780390,0.046411400,-0.02508330,0.015125200,0.02318960,-0.00741249
5,6_6,79,2,-0.01165420,0.015232400,0.01065710,-0.071689000,-0.05411840,0.00316309
6,7_7,74,2,0.00233242,0.022173500,0.02882250,0.004397390,0.07210630,-0.02177460


In [108]:
nrow(age_sex)

[1] 982

In [255]:
df = mat

In [258]:
# Seurat object
mydir = "/share/ScratchGeneral/anncuo/sc-integration-workshop/"
all_meta_filename = paste0(mydir,"all_meta_new.rds")
# all_meta_filename = paste0(mydir,"all_meta.rds")
all_meta = readRDS(all_meta_filename)
head(all_meta)

,batch_id,celltype.mDC.l1,celltype.mDC.l2,celltype.mDC.l3,donor_summary,sex,development_stage,ethnicity,disease
,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,<fct>,<fct>
10_1016_j_cell_2020_08_001_Schulte-Schrepping-0,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono,10_1016_j_cell_2020_08_001_C19-CB-0001,male,third decade human stage,unknown,COVID-19
10_1016_j_cell_2020_08_001_Schulte-Schrepping-1,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono,10_1016_j_cell_2020_08_001_C19-CB-0003,male,fifth decade human stage,unknown,COVID-19
10_1016_j_cell_2020_08_001_Schulte-Schrepping-10,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono,10_1016_j_cell_2020_08_001_C19-CB-0001,male,third decade human stage,unknown,COVID-19
10_1016_j_cell_2020_08_001_Schulte-Schrepping-100,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono,10_1016_j_cell_2020_08_001_C19-CB-0003,male,fifth decade human stage,unknown,COVID-19
10_1016_j_cell_2020_08_001_Schulte-Schrepping-1000,10_1016_j_cell_2020_08_001,CD8 T,CD8 TEM,CD8 TEM_5,10_1016_j_cell_2020_08_001_C19-CB-0001,male,third decade human stage,unknown,COVID-19
10_1016_j_cell_2020_08_001_Schulte-Schrepping-10000,10_1016_j_cell_2020_08_001,Mono,CD14 Mono,CD14 Mono,10_1016_j_cell_2020_08_001_C19-CB-0005,male,eighth decade human stage,unknown,COVID-19


In [259]:
celltypes = all_meta$celltype.mDC.l2
celltypes = gsub(" ","_",celltypes)

In [262]:
df = as.data.frame(df)
head(df)

,1_1_B intermediate,1_1_B memory,1_1_B naive,1_1_CD14 Mono,1_1_CD16 Mono,1_1_CD4 CTL,1_1_CD4 Naive,1_1_CD4 TCM,1_1_CD4 TEM,1_1_CD8 Naive,⋯,999_1000_dnT,999_1000_gdT,999_1000_HSPC,999_1000_ILC,999_1000_MAIT,999_1000_NK,999_1000_NK_CD56bright,999_1000_Plasmablast,999_1000_Platelet,999_1000_Treg
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.8,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,3.436564,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [264]:
female_donors = age_sex[age_sex$SEX==2,"sampleid"]

In [267]:
head(counts)

MIR1302-10
FAM138A
OR4F5
RP11-34P13.7
RP11-34P13.8
AL627309.1


In [ ]:
colnames(df) = gsub(" ","_",colnames(df))
for (celltype in celltypes){
    print(celltype)
    samples = colnames(df)[grep(celltype,colnames(df))]
    counts = df[,samples]
    samples = gsub(paste0("_",celltype),"",samples)
    meta_subset = age_sex[age_sex$sampleid %in% colnames(counts),]
    rownames(meta_subset) = meta_subset$sampleid
    colnames(counts) = samples
    sce = CreateSeuratObject(counts, meta.data = meta_subset)
    Idents(object = sce, cells = female_donors) <- "Female"
    mat = FindMarkers(sce, ident.1 = "Female", min.pct = 0.5)
    write.csv(mat,paste0("/share/ScratchGeneral/anncuo/sc-integration-workshop/new_DE_results_",celltype,".csv"))
}

In [269]:
head(df)

,1_1_B intermediate,1_1_B memory,1_1_B naive,1_1_CD14 Mono,1_1_CD16 Mono,1_1_CD4 CTL,1_1_CD4 Naive,1_1_CD4 TCM,1_1_CD4 TEM,1_1_CD8 Naive,⋯,999_1000_dnT,999_1000_gdT,999_1000_HSPC,999_1000_ILC,999_1000_MAIT,999_1000_NK,999_1000_NK_CD56bright,999_1000_Plasmablast,999_1000_Platelet,999_1000_Treg
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.8,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,3.436564,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [268]:
samples = colnames(df)[grep(celltype,colnames(df))]
head(samples)

character(0)

In [ ]:
    counts = df[,samples]
    samples = gsub(paste0("_",celltype),"",samples)
    meta_subset = age_sex[age_sex$sampleid %in% colnames(counts),]
    rownames(meta_subset) = meta_subset$sampleid
    colnames(counts) = samples
    sce = CreateSeuratObject(counts, meta.data = meta_subset)
    Idents(object = sce, cells = female_donors) <- "Female"
    mat = FindMarkers(sce, ident.1 = "Female", min.pct = 0.5)
    write.csv(mat,paste0("/share/ScratchGeneral/anncuo/sc-integration-workshop/new_DE_results_",celltype,".csv"))

In [228]:
# mono_cd14_samples = colnames(counts)[grep("Monocyte.CD14",colnames(counts))]
mono_other_samples = colnames(counts)[grep("Monocyte.FCGR3A",colnames(counts))]
# DC_samples = colnames(counts)[grep("Dendritic.cell",colnames(counts))]
# plasma_samples = colnames(counts)[grep("IgJ",colnames(counts))]

In [230]:
counts_mono_other = df1[,mono_other_samples]

In [204]:
Bcells_1_samples = colnames(counts)[grep("FCER2..B.cell.1",colnames(counts))]

In [221]:
counts_Bcells1 = df1[,Bcells_1_samples]
ncol(counts_Bcells1)
head(counts_Bcells1)

[1] 981

,X1_1_TCL1A..FCER2..B.cell.1,X10_10_TCL1A..FCER2..B.cell.1,X1000_1001_TCL1A..FCER2..B.cell.1,X1001_1002_TCL1A..FCER2..B.cell.1,X1002_1003_TCL1A..FCER2..B.cell.1,X1003_1004_TCL1A..FCER2..B.cell.1,X1004_1005_TCL1A..FCER2..B.cell.1,X1005_1006_TCL1A..FCER2..B.cell.1,X1006_1007_TCL1A..FCER2..B.cell.1,X1007_1008_TCL1A..FCER2..B.cell.1,⋯,X990_991_TCL1A..FCER2..B.cell.1,X991_992_TCL1A..FCER2..B.cell.1,X992_993_TCL1A..FCER2..B.cell.1,X993_994_TCL1A..FCER2..B.cell.1,X994_995_TCL1A..FCER2..B.cell.1,X995_996_TCL1A..FCER2..B.cell.1,X996_997_TCL1A..FCER2..B.cell.1,X997_998_TCL1A..FCER2..B.cell.1,X998_999_TCL1A..FCER2..B.cell.1,X999_1000_TCL1A..FCER2..B.cell.1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
RP11-34P13.7,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
RP11-34P13.8,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
AL627309.1,0,0.000000,0,0.000000,0.000000,0.000000,1.718282,1.718282,0.000000,0.000000,⋯,0.000000,3.436564,0,0.000000,0.000000,0.00000,1.718282,0.000000,1.718282,0.000000
RP11-34P13.9,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
AP006222.2,0,3.436564,0,6.873127,5.154845,3.436564,0.000000,1.718282,3.436564,5.154845,⋯,1.718282,3.436564,0,3.436564,3.436564,12.02797,16.698747,1.718282,8.591409,6.873127


In [222]:
colnames(counts_Bcells1) = gsub("X","",gsub("\\.","",gsub("_TCL1A..FCER2..B.cell.1","",colnames(counts_plasma))))
head(colnames(counts_Bcells1))
counts_Bcells1[1:5,1:5]

[1] "1_1"       "10_10"     "1000_1001" "1001_1002" "1002_1003" "1003_1004"

,1_1,10_10,1000_1001,1001_1002,1002_1003
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0
AL627309.1,0,0,0,0,0
RP11-34P13.9,0,0,0,0,0


In [157]:
# colnames(counts_DC) = gsub("X","",gsub("\\.","",gsub("_Dendritic.cell","",colnames(counts_DC))))
# counts_DC[1:5,1:5]

,1_1,10_10,1000_1001,1001_1002,1002_1003
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0,0,0,0
FAM138A,0,0,0,0,0
OR4F5,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0


In [231]:
colnames(counts_mono_other) = gsub("X","",gsub("\\.","",gsub("_Monocyte.FCGR3A","",colnames(counts_mono_other))))
counts_mono_other[1:5,1:5]

,1_1,10_10,1000_1001,1001_1002,1002_1003
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-10,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0
AL627309.1,0,0,0,0,0
RP11-34P13.9,0,0,0,0,0


In [158]:
# colnames(counts_mono_cd14) = gsub("X","",gsub("\\.","",gsub("_Monocyte.CD14","",colnames(counts_mono_cd14))))
# counts_mono_cd14[1:5,1:5]

In [232]:
nrow(age_sex)
meta_subset = age_sex[age_sex$sampleid %in% colnames(counts_mono_other),]
rownames(meta_subset) = meta_subset$sampleid
nrow(meta_subset)
head(meta_subset)

[1] 982

[1] 933

,sampleid,age,SEX,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6
,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1_1,1_1,65,2,0.02775250,-0.000592955,0.00757785,-0.014092900,0.00932333,0.03137540
2_2,2_2,48,2,-0.04813530,-0.056674100,-0.01547420,-0.000729391,0.00027473,0.01116000
3_3,3_3,53,2,0.01369180,-0.046308100,-0.00584410,0.026565000,-0.00435940,0.03093290
4_4,4_4,48,2,0.01780390,0.046411400,-0.02508330,0.015125200,0.02318960,-0.00741249
6_6,6_6,79,2,-0.01165420,0.015232400,0.01065710,-0.071689000,-0.05411840,0.00316309
7_7,7_7,74,2,0.00233242,0.022173500,0.02882250,0.004397390,0.07210630,-0.02177460


In [224]:
length(colnames(counts_mono_cd14))

[1] 968

In [227]:
# rownames(counts_Bcells1)

In [233]:
sce_mono1 = CreateSeuratObject(counts_mono_other, meta.data = meta_subset)

In [164]:
female_donors = age_sex[age_sex$SEX==2,"sampleid"]
length(female_donors)
head(female_donors)

[1] 565

[1] "1_1" "2_2" "3_3" "4_4" "6_6" "7_7"

In [234]:
Idents(object = sce_mono1, cells = female_donors) <- "Female"

In [235]:
mat = FindMarkers(sce_mono1, ident.1 = "Female", min.pct = 0.5)

In [236]:
# mat_DC = mat
# mat_mono_CD14 = mat
# mat_plasma = mat
# mat_mono_FGFRA = mat

In [237]:
head(mat_mono_FGFRA)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EIF1AY,2.041688e-103,-Inf,0.128,0.797,5.756744e-99
XIST,5.764760e-102,Inf,0.957,0.490,1.625432e-97
DDX3Y,1.251870e-80,-186.80357,0.145,0.718,3.529774e-76
UTY,3.524454e-78,-39.83605,0.063,0.617,9.937551e-74
NRAS,5.571920e-05,12.44259,0.440,0.576,1.000000e+00
ATF7IP2,7.527916e-05,-17.31147,0.516,0.629,1.000000e+00


In [194]:
head(mat_plasma)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EIF1AY,3.186238e-100,-Inf,0.100,0.829,1.043111e-95
XIST,2.670841e-74,Inf,0.904,0.374,8.743798e-70
KDM5D,3.496920e-65,-863.33348,0.060,0.609,1.144822e-60
JPX,4.999232e-06,-28.28508,0.624,0.490,1.636649e-01
ZRSR2,1.388235e-03,122.65216,0.564,0.487,1.000000e+00
SAT2,4.469907e-03,-578.98537,0.787,0.742,1.000000e+00


In [183]:
head(mat_mono_CD14)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DDX3Y,1.884420e-101,-Inf,0.285,0.869,6.169215e-97
EIF1AY,2.525529e-100,-Inf,0.248,0.852,8.268076e-96
UTY,4.276327e-71,-68.67160,0.142,0.664,1.399984e-66
ZFY,3.324473e-64,-41.65362,0.045,0.516,1.088366e-59
ARSD,6.514707e-05,-38.28189,0.673,0.569,1.000000e+00
XPO7,6.998734e-05,-28.18894,0.539,0.655,1.000000e+00


In [239]:
head(mat_DC,14)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
EIF1AY,3.712014e-123,-591.374289,0.124,0.861,1.215239e-118
XIST,1.198056e-116,Inf,0.977,0.504,3.922196e-112
DDX3Y,9.434714e-106,-550.992393,0.153,0.818,3.088737e-101
UTY,1.812252e-71,-42.523526,0.045,0.552,5.932952e-67
KDM5D,3.250923e-58,-22.556621,0.061,0.509,1.064287e-53
TTTY15,7.772480e-50,-21.789750,0.086,0.504,2.544554e-45
WASH4P,2.414997e-08,-Inf,0.871,0.927,7.906218e-04
SRSF6,8.059152e-08,-Inf,0.865,0.917,2.638405e-03
PLEKHF2,4.884295e-07,-8.633659,0.530,0.681,1.599020e-02


In [238]:
nrow(mat)
nrow(mat[mat$p_val_adj<0.05,])

[1] 4460

[1] 4

In [ ]:
dds <- DESeqDataSetFromMatrix(cluster_counts, 
                              colData = cluster_metadata, 
                              design = ~ group_id)

In [ ]:
dds <- DESeq(dds)